Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [192]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from tqdm import tqdm
import time

First reload the data we generated in `1_notmnist.ipynb`.

In [31]:
pickle_file = './input/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [32]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [33]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [73]:
def run_model(logReg=True, batch_size=128, hidden_nodes=1024, beta=1e-2, num_epoch=6, num_steps=500,
             restrict_train=False, dropout = False, dropout_rate=0.5):
    graph = tf.Graph() # Init graph.

    # Describe the computation: what the inputs, the variables, and the operations look like. 
    # Create them as nodes over a computation graph.
    with graph.as_default():

        # Input data. For the training data, we use a placeholder that will be fed
        # at run time with a training minibatch.
        tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)

        if logReg:
            hidden_nodes = num_labels

        weights_1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_nodes]))
        biases_1 = tf.Variable(tf.zeros([hidden_nodes]))

        weights_2 = tf.Variable(
        tf.truncated_normal([hidden_nodes, num_labels]))
        biases_2 = tf.Variable(tf.zeros([num_labels]))

        reg_loss = beta*tf.nn.l2_loss(weights_1)
        # Training computation.
        # Lambda function to compute the output depending on if we're doing Logistic regression or not.
        if not logReg:
            if dropout: # If dropout, apply to hidden layer inputs.
                out2 = lambda x:  tf.matmul(
                    tf.nn.dropout(tf.nn.relu(tf.matmul(x, weights_1) + biases_1), dropout_rate), weights_2) + biases_2
            out = lambda x:  tf.matmul(tf.nn.relu(tf.matmul(x, weights_1) + biases_1), weights_2) + biases_2
            reg_loss+= beta*tf.nn.l2_loss(weights_2)
        else:
            out = lambda x: tf.matmul(x, weights_1) + biases_1

        if dropout:
            output_drop = out2(tf_train_dataset) # Output from batch of training set data, with dropout.
            output = out(tf_train_dataset) # Output from batch of training set data, without dropout.
            loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=output_drop)) + reg_loss # The loss.
        else:
            output = out(tf_train_dataset) # Output from batch of training set data.
            loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=output)) + reg_loss # The loss.

        # Optimizer.
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

        # Predictions for the training, validation, and test data.
        train_prediction = tf.nn.softmax(output) # Probabilities of classes given training batch.
        valid_prediction = tf.nn.softmax(out(tf_valid_dataset)) # Probabilities of classes given validation set.
        test_prediction = tf.nn.softmax(out(tf_test_dataset))
        # Probabilities of classes given given test set

    with tf.Session(graph=graph) as session:
        print("Number of epochs: ", num_epoch) 
        print("Steps in each epoch: ", num_steps)
        tf.global_variables_initializer().run() # Ensures the parameters get initialized as specified.
        print("Initialized")
        time.sleep(1) # Sleep to prevent progressbar to "leak"
        # Iterate over epochs.
        for i in range(num_epoch+1):
            print("="*45 + " Epoch: " + str(i) + " " + "="*45)
            time.sleep(1)
            for step in tqdm(range(num_steps), total=num_steps):
                # Randomly sample a batch.
                if restrict_train: # If restrict train, use only 10 batches of training set.
                    permutation = np.random.randint(0, 10*batch_size, batch_size)
                else:
                    permutation = np.random.randint(0,train_labels.shape[0], batch_size)
                batch_data = train_dataset[permutation,:]
                batch_labels = train_labels[permutation]

                # Define dictionary telling the session where to feed the minibatch.
                # The key of the dictionary is the placeholder node of the graph to be fed,
                # and the value is the numpy array to feed to it.
                feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}

                # Run the operations on this graph 
                _, l, predictions = session.run(
                  [optimizer, loss, train_prediction], feed_dict=feed_dict)

            print("Minibatch loss: %f" % (l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            time.sleep(1)

        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

In [69]:
# Run logistic regression.
run_model(logReg=True, beta=1e-3)

Number of epochs:  6
Steps in each epoch:  500
Initialized
============================================= Epoch: 0 =============================================


100%|██████████| 500/500 [00:00<00:00, 1059.88it/s]


Minibatch loss: 2.751889
Minibatch accuracy: 78.9%
Validation accuracy: 76.6%
============================================= Epoch: 1 =============================================


100%|██████████| 500/500 [00:00<00:00, 1108.46it/s]


Minibatch loss: 1.438471
Minibatch accuracy: 83.6%
Validation accuracy: 78.4%
============================================= Epoch: 2 =============================================


...

Minibatch loss: 0.783925
Minibatch accuracy: 85.2%
Validation accuracy: 81.6%
============================================= Epoch: 5 =============================================


100%|██████████| 500/500 [00:00<00:00, 1076.10it/s]


Minibatch loss: 0.695848
Minibatch accuracy: 87.5%
Validation accuracy: 82.1%
============================================= Epoch: 6 =============================================


100%|██████████| 500/500 [00:00<00:00, 1109.87it/s]


Minibatch loss: 0.598026
Minibatch accuracy: 86.7%
Validation accuracy: 82.3%
Test accuracy: 88.7%


The logistic regression model without regularization got an test accuracy of 86.6%, so an improvement.

In [70]:
# Run two layer network.
run_model(logReg=False, beta=1e-3)

Number of epochs:  6
Steps in each epoch:  500
Initialized
============================================= Epoch: 0 =============================================


100%|██████████| 500/500 [00:05<00:00, 91.66it/s]


Minibatch loss: 194.719833
Minibatch accuracy: 81.2%
Validation accuracy: 81.4%
============================================= Epoch: 1 =============================================


100%|██████████| 500/500 [00:05<00:00, 92.05it/s]


Minibatch loss: 114.523514
Minibatch accuracy: 82.0%
Validation accuracy: 82.3%
============================================= Epoch: 2 =============================================


...

Minibatch loss: 25.188179
Minibatch accuracy: 85.2%
Validation accuracy: 86.5%
============================================= Epoch: 5 =============================================


100%|██████████| 500/500 [00:05<00:00, 92.95it/s]


Minibatch loss: 15.472729
Minibatch accuracy: 88.3%
Validation accuracy: 86.8%
============================================= Epoch: 6 =============================================


100%|██████████| 500/500 [00:05<00:00, 92.26it/s]


Minibatch loss: 9.539312
Minibatch accuracy: 88.3%
Validation accuracy: 87.6%
Test accuracy: 93.4%


The two layer network without regularization got an test accuracy of 89.3%, so a huge improvement.

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

With the training set size reduced to 10 batches.

In [71]:
run_model(logReg=False, beta=1e-3, restrict_train=True)

Number of epochs:  6
Steps in each epoch:  500
Initialized
============================================= Epoch: 0 =============================================


100%|██████████| 500/500 [00:05<00:00, 91.33it/s]


Minibatch loss: 190.999390
Minibatch accuracy: 99.2%
Validation accuracy: 76.7%
============================================= Epoch: 1 =============================================


100%|██████████| 500/500 [00:05<00:00, 88.74it/s]


Minibatch loss: 115.702850
Minibatch accuracy: 100.0%
Validation accuracy: 76.8%
============================================= Epoch: 2 =============================================


...

Minibatch loss: 25.807690
Minibatch accuracy: 100.0%
Validation accuracy: 77.0%
============================================= Epoch: 5 =============================================


100%|██████████| 500/500 [00:05<00:00, 92.34it/s]


Minibatch loss: 15.652592
Minibatch accuracy: 100.0%
Validation accuracy: 77.5%
============================================= Epoch: 6 =============================================


100%|██████████| 500/500 [00:05<00:00, 89.81it/s]


Minibatch loss: 9.499249
Minibatch accuracy: 100.0%
Validation accuracy: 78.2%
Test accuracy: 85.8%


Extreme overfit, 100% accuracy on the training data. Whereas the validation accuracy barely increases, probably an awful classifier when applied to new data.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [75]:
run_model(logReg=False, beta=1e-3, restrict_train=True, dropout=True, dropout_rate = 0.8)

Number of epochs:  6
Steps in each epoch:  500
Initialized
============================================= Epoch: 0 =============================================


100%|██████████| 500/500 [00:05<00:00, 89.77it/s]


Minibatch loss: 190.632553
Minibatch accuracy: 98.4%
Validation accuracy: 78.0%
============================================= Epoch: 1 =============================================


100%|██████████| 500/500 [00:05<00:00, 89.18it/s]


Minibatch loss: 115.617126
Minibatch accuracy: 100.0%
Validation accuracy: 78.9%
============================================= Epoch: 2 =============================================


...

Minibatch loss: 25.786812
Minibatch accuracy: 100.0%
Validation accuracy: 79.5%
============================================= Epoch: 5 =============================================


100%|██████████| 500/500 [00:05<00:00, 86.32it/s]


Minibatch loss: 15.639150
Minibatch accuracy: 100.0%
Validation accuracy: 79.6%
============================================= Epoch: 6 =============================================


100%|██████████| 500/500 [00:05<00:00, 84.81it/s]


Minibatch loss: 9.488671
Minibatch accuracy: 100.0%
Validation accuracy: 79.8%
Test accuracy: 86.9%


Extreme overfit, 100% accuracy on the training data. However the validation accuracy do increase, which implies that the model is actually learning something.

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


As a first try lets use our model with all the training data, regularization and dropout a go for more than 6 epochs

In [76]:
run_model(logReg=False, beta=1e-3, restrict_train=False, dropout=True, dropout_rate = 0.5, num_epoch=15)

Number of epochs:  15
Steps in each epoch:  500
Initialized
============================================= Epoch: 0 =============================================


100%|██████████| 500/500 [00:06<00:00, 80.10it/s]


Minibatch loss: 196.761414
Minibatch accuracy: 82.0%
Validation accuracy: 80.9%
============================================= Epoch: 1 =============================================


100%|██████████| 500/500 [00:06<00:00, 83.81it/s]


Minibatch loss: 119.747803
Minibatch accuracy: 77.3%
Validation accuracy: 80.9%
============================================= Epoch: 2 =============================================


...

Minibatch loss: 0.881037
Minibatch accuracy: 88.3%
Validation accuracy: 87.4%
============================================= Epoch: 14 =============================================


100%|██████████| 500/500 [00:06<00:00, 82.89it/s]


Minibatch loss: 0.574905
Minibatch accuracy: 92.2%
Validation accuracy: 87.6%
============================================= Epoch: 15 =============================================


100%|██████████| 500/500 [00:05<00:00, 86.05it/s]


Minibatch loss: 0.820527
Minibatch accuracy: 85.2%
Validation accuracy: 87.6%
Test accuracy: 93.3%


Fairly good, but lets try to improve it using more hidden layers as well as momentum.

In [200]:
batch_size=128 # Batch size.
hidden_nodes=300 # Number of hidden nodes in the hidden layers.
beta=1e-3 # Regularization parameter.
num_epoch=10 # Number of epochs.
num_steps=4000 # Number of steps in each epoch.
dropout_rate=0.5 # Dropout rate.
layers = 7 # Number of layers
stdvinit = 0.05 # Standard deviation of the weight initialisation, we have to use a small std if we're using alot of
                # layers since the gradient goes crazy otherwise.

graph = tf.Graph() # Init graph.

# Describe the computation: what the inputs, the variables, and the operations look like. 
# Create them as nodes over a computation graph.
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Empty lists to store weights and biases for the layers.
    weights = []
    biases = []
    for h in range(layers):
        if h == 0: # If first layers, create one that takes the inputs.
            weights.append(tf.Variable(
            tf.truncated_normal([image_size * image_size, hidden_nodes],stddev=stdvinit)))
            biases.append(tf.Variable(tf.zeros([hidden_nodes])))
            reg_loss = beta*tf.nn.l2_loss(weights[h]) # Add regularization.
        elif h == layers-1: # If output layer.
            weights.append(tf.Variable(tf.truncated_normal([hidden_nodes, num_labels],stddev=stdvinit)))
            biases.append(tf.Variable(tf.zeros([num_labels])))
        else: # If hidden layer.
            weights.append(tf.Variable(
            tf.truncated_normal([hidden_nodes, hidden_nodes],stddev=stdvinit)))
            biases.append(tf.Variable(tf.zeros([hidden_nodes])))
            reg_loss += beta*tf.nn.l2_loss(weights[h])

    # Compute output when training.
    def out_train(x):
        out = tf.nn.relu(tf.matmul(x, weights[0]) + biases[0]) # Output of first layer.
        for k,(weight,bias) in enumerate(zip(weights,biases)):
            if k == 0:
                pass
            elif k == len(weights)-1: # Output from output layer, with dropout.
                return tf.matmul(tf.nn.dropout(out, dropout_rate), weight) + bias
            else: # Output from hidden layer, with dropout. 
                out = tf.nn.relu(tf.matmul(tf.nn.dropout(out, dropout_rate), weight) + bias)
    
    # Compute output when not training, without dropout.
    def out(x):
        out = tf.nn.relu(tf.matmul(x, weights[0]) + biases[0])
        for k,(weight,bias) in enumerate(zip(weights,biases)):
            if k == 0:
                pass
            elif k == len(weights)-1:
                return tf.matmul(out, weight) + bias
            else:
                out = tf.nn.relu(tf.matmul(out, weight) + bias)        
    
    logit = out_train(tf_train_dataset) # Output when training.
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits( 
        labels=tf_train_labels, logits=logit)) + reg_loss # The loss.

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, 1000, 0.9) # Learning rate decay:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    #optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(out(tf_train_dataset)) # Probabilities of classes given training batch.
    valid_prediction = tf.nn.softmax(out(tf_valid_dataset)) # Probabilities of classes given validation set.
    test_prediction = tf.nn.softmax(out(tf_test_dataset))
    # Probabilities of classes given given test set

with tf.Session(graph=graph) as session:
    print("Number of epochs: ", num_epoch) 
    print("Steps in each epoch: ", num_steps)
    print("Number of layers: ", layers)
    tf.global_variables_initializer().run() # Ensures the parameters get initialized as specified.
    print("Initialized")
    time.sleep(1.5) # Sleep to prevent progressbar to "leak"
    # Iterate over epochs.
    for i in range(num_epoch+1):
        print("="*45 + " Epoch: " + str(i) + " " + "="*45)
        time.sleep(1.5)
        for step in tqdm(range(num_steps), total=num_steps):
            permutation = np.random.randint(0,train_labels.shape[0], batch_size)
            batch_data = train_dataset[permutation,:]
            batch_labels = train_labels[permutation]

            # Define dictionary telling the session where to feed the minibatch.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}

            # Run the operations on this graph 
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)

        print("Minibatch loss: %f" % (l))
        print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
        print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        time.sleep(1)

    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Number of epochs:  10
Steps in each epoch:  4000
Number of layers:  7
Initialized
============================================= Epoch: 0 =============================================


100%|██████████| 4000/4000 [01:02<00:00, 63.81it/s]


Minibatch loss: 0.725452
Minibatch accuracy: 86.7%
Validation accuracy: 85.7%
============================================= Epoch: 1 =============================================


100%|██████████| 4000/4000 [01:02<00:00, 63.99it/s]


Minibatch loss: 0.501557
Minibatch accuracy: 91.4%
Validation accuracy: 87.4%
============================================= Epoch: 2 =============================================


...

Minibatch loss: 0.532219
Minibatch accuracy: 89.8%
Validation accuracy: 90.2%
============================================= Epoch: 9 =============================================


100%|██████████| 4000/4000 [01:08<00:00, 58.20it/s]


Minibatch loss: 0.354487
Minibatch accuracy: 92.2%
Validation accuracy: 90.3%
============================================= Epoch: 10 =============================================


100%|██████████| 4000/4000 [01:08<00:00, 58.91it/s]


Minibatch loss: 0.380115
Minibatch accuracy: 94.5%
Validation accuracy: 90.4%
Test accuracy: 95.6%


You could probably tweak the hyperparameters and get a better accuracy, I did not spend time on this though.